First, load the 311 complaints dataset available here https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9

In [1]:
CSV_PATH = './data/311.csv'
import subprocess
from tqdm import tqdm
import pandas as pd 
import os
from h3 import h3
import warnings
warnings.filterwarnings('ignore')


def file_len(fname):
    p = subprocess.Popen(['wc', '-l', fname], stdout=subprocess.PIPE, 
                                              stderr=subprocess.PIPE)
    result, err = p.communicate()
    if p.returncode != 0:
        raise IOError(err)
    return int(result.strip().split()[0])+1

n_rows = file_len(CSV_PATH)
print (f'Exact number of rows: {n_rows}')
df_tmp = pd.read_csv(CSV_PATH, nrows=5)
df_tmp.head()


Exact number of rows: 27382105


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,27797840,04/07/2014 04:18:00 PM,04/08/2014 11:33:00 AM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,11229.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.597262,-73.951006,"(40.59726241863315, -73.95100611242795)"
1,34969370,12/09/2016 10:14:44 PM,12/09/2016 10:15:17 PM,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,Food Stamp,NYC Street Address,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34969353,12/07/2016 08:59:03 AM,NaN,DOT,Department of Transportation,Street Condition,Wear & Tear,NaN,11233.0,HANCOCK STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34969558,12/09/2016 03:37:25 PM,12/12/2016 11:33:02 AM,DOF,Personal Exemption Unit,DOF Property - Reduction Issue,Personal STAR Exemption,"1-, 2- and 3- Family Home",11370.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34969585,12/08/2016 03:16:00 PM,12/09/2016 12:00:00 PM,DSNY,BCC - Staten Island,Overflowing Litter Baskets,6 Overflowing Litter Baskets,Sidewalk,NaN,MAGUIRE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
types = {
    #'Unique Key': 'float32',
              'Complaint Type': 'category', 
              'Longitude': 'float32',
              'Latitude': 'float32',
#              'Incident Zip': 'int',
              'Created Date': 'str',
#              'Closed Date': 'str',

        }
cols = list(types.keys())
chunksize = 10_000_000

df_list = [] # list to hold the batch dataframe

def process_date(df_chunk, key):
    df_chunk[key] = df_chunk[key].str.slice(0, 16)
    #df_chunk[key] = pd.to_datetime(df_chunk[key], utc=True, format='%Y-%m-%d %H:%M')
    
for df_chunk in tqdm(pd.read_csv(CSV_PATH, usecols=cols, dtype=types, chunksize=chunksize)):
    # Neat trick from https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
    # Using parse_dates would be much slower!
#    process_date(df_chunk, 'Closed Date')
#    process_date(df_chunk, 'Created Date')
    df_list.append(df_chunk) 

    
pdf = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what we have loaded
pdf.info()
#takes 1:30 minutes

3it [00:57, 19.03s/it]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27382103 entries, 0 to 27382102
Data columns (total 4 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Created Date    object 
 1   Complaint Type  object 
 2   Latitude        float32
 3   Longitude       float32
dtypes: float32(2), object(2)
memory usage: 626.7+ MB


In [3]:
pd.options.display.max_rows = 1000

counts = pdf['Complaint Type'].value_counts()
print(counts)



Noise - Residential                              2575599
HEAT/HOT WATER                                   1589587
Illegal Parking                                  1424426
Blocked Driveway                                 1188263
Street Condition                                 1090368
Street Light Condition                           1035400
HEATING                                           887869
Noise - Street/Sidewalk                           863368
PLUMBING                                          780618
Water System                                      739604
UNSANITARY CONDITION                              570281
Noise                                             564961
General Construction/Plumbing                     563138
GENERAL CONSTRUCTION                              500863
Traffic Signal Condition                          500226
Noise - Commercial                                446898
Sewer                                             419190
PAINT/PLASTER                  

In [6]:
APERTURE_SIZE = 10
hex_col = 'hex'+str(APERTURE_SIZE)


pdf[hex_col] = 0
import json

print(1)
def binRow(df311):
    # find hexs containing the points
    df311[hex_col] = df311.apply(lambda x: h3.geo_to_h3(x.Latitude,x.Longitude,APERTURE_SIZE),1)

    # aggregate the points
    df311g = df311.groupby(hex_col).size()#.to_frame('cnt').reset_index()

    #find center of hex for visualization
#     df311g['lat'] = df311g[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
#     df311g['lng'] = df311g[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])
    return (df311g)


def saveComplaint(complaint):
    print(complaint)
    data = pdf.loc[pdf['Complaint Type'] ==(complaint)]
    binned = binRow(data)
    complaint = complaint.replace(' ', '-').replace('/','-')
    hi = [(hex, count)  for hex,count in binned.iteritems()]
    with open('./data/'+complaint+'.json', 'w') as outfile: json.dump(hi, outfile)
    
    

stuff = [
    'Noise - Residential',                         
#'HEAT/HOT WATER' ,                              
#'Street Condition'   ,                           
#'Illegal Parking ',                              
#'Blocked Driveway'  ,                            
#'Street Light Condition' ,                       
#'HEATING'          ,                             
#'PLUMBING'     ,                                 
#'Water System'  ,                                
#'Noise - Street/Sidewalk' ,                      
# 'GENERAL CONSTRUCTION'  ,                                                                
#'UNSANITARY CONDITION'
        ]                          


    
    
import csv, json
from geojson import Feature, FeatureCollection, Point, dump


    
def saveComplaint2(complaint):
    
    data = pdf.loc[pdf['Complaint Type'] ==(complaint)]
    binned = binRow(data)
    complaint = complaint.replace(' ', '-').replace('-','')
    print(complaint)
    #[(hex, count)  for hex,count in binned.iteritems()]
    features = []
    for key,val in binned.iteritems():
        
        print(key,val)
        if h3.string_to_h3(key) == 0x0: continue
        else: 
            lat,long = h3.h3_to_geo(key)
            #print(lat, long)
            features.append(
                Feature(
                    geometry = Point((long, lat)),
                    properties = {
                        'noise_value': val,
                        #'created_at': 
                    }
                )
            )
    #return
    print(features[0])
    collection = FeatureCollection(features)
    with open('./data/' + complaint + ".geojson", "w") as f:
        print ('writing to ' + complaint + ".geojson",)
        #f.write('%s' % collection)
        dump(collection, f)

    #with open('./data/'+complaint+'.json', 'w') as outfile: json.dump(hi, outfile)
 
    #     with open(complaint + '.csv', 'w') as csvfile:
    #         reader = csv.reader(csvfile, delimiter=',')
    #         for latitude, longitude, weather, temp in reader:
    #             latitude, longitude = map(float, (latitude, longitude))
          

   

        
for complaint in stuff:
    saveComplaint2(complaint)

1
NoiseResidential
0 7841
8a2a1000005ffff 1
8a2a10000097fff 1
8a2a10000297fff 2
8a2a100002d7fff 3
8a2a1000044ffff 18
8a2a1000045ffff 3
8a2a1000046ffff 5
8a2a10000607fff 1
8a2a10000627fff 16
8a2a1000062ffff 43
8a2a10000637fff 131
8a2a10000647fff 10
8a2a1000064ffff 2
8a2a1000065ffff 3
8a2a10000667fff 3
8a2a1000066ffff 8
8a2a10000677fff 45
8a2a10000717fff 27
8a2a1000075ffff 1
8a2a100007a7fff 3
8a2a100007affff 7
8a2a10002417fff 16
8a2a10002427fff 12
8a2a100024a7fff 8
8a2a100024affff 4
8a2a100024b7fff 2
8a2a100024c7fff 1
8a2a1000250ffff 3
8a2a10002537fff 101
8a2a10002587fff 6
8a2a1000258ffff 34
8a2a10002597fff 1
8a2a100025a7fff 24
8a2a100025affff 2
8a2a100025b7fff 11
8a2a10002617fff 12
8a2a1000264ffff 1
8a2a100026dffff 1
8a2a1000291ffff 1
8a2a10002a07fff 5
8a2a10002c17fff 1
8a2a10002c2ffff 1
8a2a10002cf7fff 2
8a2a10003807fff 2
8a2a1000380ffff 1
8a2a10003817fff 43
8a2a1000381ffff 6
8a2a10003827fff 23
8a2a1000382ffff 3
8a2a10003837fff 3
8a2a10003847fff 14
8a2a1000384ffff 13
8a2a1000385ffff 3


8a2a10014b57fff 1
8a2a10014b8ffff 2
8a2a10014c07fff 99
8a2a10014c0ffff 125
8a2a10014c17fff 11
8a2a10014c1ffff 15
8a2a10014c27fff 101
8a2a10014c2ffff 9
8a2a10014c37fff 10
8a2a10014c47fff 3
8a2a10014c4ffff 36
8a2a10014c57fff 75
8a2a10014c5ffff 27
8a2a10014c67fff 8
8a2a10014c77fff 15
8a2a10014c87fff 3
8a2a10014c97fff 16
8a2a10014c9ffff 8
8a2a10014ca7fff 3
8a2a10014cb7fff 45
8a2a10014cc7fff 19
8a2a10014ccffff 52
8a2a10014cd7fff 7
8a2a10014cdffff 41
8a2a10014ce7fff 11
8a2a10014ceffff 51
8a2a10014cf7fff 2
8a2a10014d0ffff 49
8a2a10014d17fff 14
8a2a10014d1ffff 97
8a2a10014d2ffff 21
8a2a10014d47fff 7
8a2a10014d4ffff 47
8a2a10014d57fff 35
8a2a10014d5ffff 61
8a2a10014d6ffff 7
8a2a10014d77fff 67
8a2a10014d87fff 39
8a2a10014d8ffff 10
8a2a10014d97fff 340
8a2a10014d9ffff 40
8a2a10014da7fff 38
8a2a10014daffff 127
8a2a10014db7fff 576
8a2a10015007fff 5
8a2a10015017fff 7
8a2a1001501ffff 15
8a2a10015037fff 17
8a2a10015047fff 3
8a2a1001504ffff 4
8a2a1001505ffff 12
8a2a10015077fff 65
8a2a10015087fff 12
8a2a

8a2a1001ed9ffff 97
8a2a1001eda7fff 41
8a2a1001edaffff 77
8a2a1001edb7fff 21
8a2a100326dffff 1
8a2a10033607fff 1
8a2a1003361ffff 2
8a2a10033627fff 4
8a2a1003362ffff 9
8a2a10033647fff 2
8a2a10033667fff 37
8a2a1003366ffff 3
8a2a10033677fff 27
8a2a1003368ffff 9
8a2a1003369ffff 1
8a2a100336c7fff 4
8a2a100336cffff 3
8a2a100336d7fff 2
8a2a100336effff 1
8a2a100336f7fff 2
8a2a1003374ffff 4
8a2a10033757fff 3
8a2a1003375ffff 9
8a2a10042087fff 7
8a2a1004208ffff 3
8a2a10042097fff 4
8a2a1004209ffff 11
8a2a100420a7fff 3
8a2a100420b7fff 4
8a2a10042187fff 4
8a2a10042197fff 5
8a2a1004219ffff 12
8a2a100421a7fff 8
8a2a10042407fff 4
8a2a1004240ffff 1
8a2a1004241ffff 5
8a2a10042427fff 7
8a2a1004242ffff 8
8a2a10042437fff 2
8a2a10042447fff 16
8a2a1004244ffff 11
8a2a10042457fff 4
8a2a1004245ffff 3
8a2a10042467fff 12
8a2a1004246ffff 4
8a2a10042477fff 8
8a2a10042487fff 1
8a2a1004248ffff 4
8a2a1004249ffff 6
8a2a100424a7fff 3
8a2a100424affff 2
8a2a100424b7fff 4
8a2a100424c7fff 11
8a2a100424cffff 2
8a2a100424d7fff 

8a2a1005812ffff 2
8a2a10058137fff 6
8a2a10058157fff 2
8a2a1005815ffff 2
8a2a10058187fff 9
8a2a1005818ffff 10
8a2a10058197fff 13
8a2a1005819ffff 16
8a2a100581a7fff 18
8a2a100581affff 1
8a2a100581b7fff 11
8a2a10058207fff 1
8a2a1005820ffff 4
8a2a10058217fff 5
8a2a1005821ffff 7
8a2a10058227fff 1
8a2a10058237fff 5
8a2a10058247fff 2
8a2a10058257fff 2
8a2a10058287fff 1
8a2a1005828ffff 8
8a2a10058297fff 1
8a2a1005829ffff 2
8a2a100582affff 3
8a2a100582b7fff 4
8a2a100582c7fff 3
8a2a100582cffff 10
8a2a100582d7fff 2
8a2a100582dffff 1
8a2a100582e7fff 18
8a2a100582effff 4
8a2a100582f7fff 5
8a2a10058307fff 5
8a2a1005830ffff 11
8a2a10058317fff 1
8a2a1005831ffff 16
8a2a10058337fff 6
8a2a10058387fff 2
8a2a1005838ffff 6
8a2a10058397fff 3
8a2a1005839ffff 3
8a2a100583a7fff 3
8a2a100583affff 22
8a2a100583b7fff 3
8a2a10058407fff 5
8a2a1005840ffff 48
8a2a10058417fff 16
8a2a1005841ffff 38
8a2a10058427fff 50
8a2a1005842ffff 15
8a2a10058437fff 5
8a2a10058447fff 7
8a2a1005844ffff 10
8a2a10058457fff 17
8a2a1005845

8a2a10089967fff 68
8a2a1008996ffff 1
8a2a10089977fff 55
8a2a10089987fff 133
8a2a1008998ffff 22
8a2a10089997fff 16
8a2a1008999ffff 11
8a2a100899a7fff 185
8a2a100899affff 219
8a2a100899b7fff 365
8a2a10089a07fff 184
8a2a10089a0ffff 28
8a2a10089a17fff 88
8a2a10089a1ffff 97
8a2a10089a27fff 337
8a2a10089a2ffff 52
8a2a10089a37fff 146
8a2a10089a47fff 28
8a2a10089a4ffff 141
8a2a10089a57fff 145
8a2a10089a5ffff 214
8a2a10089a67fff 2
8a2a10089a6ffff 1
8a2a10089a77fff 481
8a2a10089a87fff 85
8a2a10089a8ffff 738
8a2a10089a97fff 292
8a2a10089a9ffff 101
8a2a10089aa7fff 136
8a2a10089aaffff 268
8a2a10089ab7fff 71
8a2a10089ac7fff 245
8a2a10089acffff 356
8a2a10089ad7fff 121
8a2a10089adffff 105
8a2a10089ae7fff 129
8a2a10089aeffff 127
8a2a10089af7fff 18
8a2a10089b07fff 640
8a2a10089b0ffff 2
8a2a10089b17fff 45
8a2a10089b1ffff 97
8a2a10089b27fff 410
8a2a10089b2ffff 12
8a2a10089b37fff 682
8a2a10089b47fff 29
8a2a10089b4ffff 14
8a2a10089b57fff 133
8a2a10089b5ffff 211
8a2a10089b6ffff 7
8a2a10089b77fff 23
8a2a10089

...

8a2a1062bccffff 1
8a2a1062bcd7fff 6
8a2a1062bcdffff 1
8a2a1062bcf7fff 7
8a2a1062bd07fff 5
8a2a1062bd0ffff 3
8a2a1062bd1ffff 3
8a2a1062bd27fff 15
8a2a1062bd2ffff 9
8a2a1062bd37fff 2
8a2a1062bd4ffff 1
8a2a1062bd57fff 2
8a2a1062bd5ffff 1
8a2a1062bd6ffff 4
8a2a1062bd77fff 3
8a2a1062bd87fff 3
8a2a1062bd97fff 3
8a2a1062bda7fff 4
8a2a1062bdaffff 15
8a2a1062bdb7fff 1
8a2a1062c007fff 9
8a2a1062c00ffff 11
8a2a1062c027fff 4
8a2a1062c02ffff 8
8a2a1062c037fff 2
8a2a1062c047fff 1
8a2a1062c04ffff 29
8a2a1062c057fff 2
8a2a1062c05ffff 338
8a2a1062c067fff 19
8a2a1062c06ffff 33
8a2a1062c077fff 5
8a2a1062c087fff 2
8a2a1062c09ffff 7
8a2a1062c0c7fff 197
8a2a1062c0cffff 309
8a2a1062c0dffff 74
8a2a1062c0e7fff 5
8a2a1062c0effff 160
8a2a1062c107fff 22
8a2a1062c10ffff 2
8a2a1062c117fff 3
8a2a1062c127fff 83
8a2a1062c12ffff 2
8a2a1062c147fff 5
8a2a1062c14ffff 45
8a2a1062c157fff 2
8a2a1062c15ffff 3
8a2a1062c167fff 3
8a2a1062c16ffff 9
8a2a1062c177fff 1
8a2a1062c1a7fff 1
8a2a1062c1affff 1
8a2a1062c1b7fff 4
8a2a1062c2

8a2a1072d78ffff 8
8a2a1072d7a7fff 5
8a2a1072d7affff 5
8a2a1072d837fff 15
8a2a1072d8a7fff 82
8a2a1072d8b7fff 773
8a2a1072d8d7fff 2
8a2a1072d917fff 36
8a2a1072d987fff 153
8a2a1072d98ffff 18
8a2a1072d997fff 762
8a2a1072d99ffff 132
8a2a1072d9a7fff 51
8a2a1072d9affff 12
8a2a1072d9b7fff 16
8a2a1072da97fff 23
8a2a1072da9ffff 26
8a2a1072dab7fff 21
8a2a1072dad7fff 1
8a2a1072db1ffff 1
8a2a1072dc07fff 118
8a2a1072dc0ffff 387
8a2a1072dc17fff 117
8a2a1072dc1ffff 549
8a2a1072dc27fff 79
8a2a1072dc2ffff 87
8a2a1072dc37fff 103
8a2a1072dc47fff 129
8a2a1072dc57fff 24
8a2a1072dc67fff 28
8a2a1072dc6ffff 4
8a2a1072dc77fff 220
8a2a1072dc87fff 58
8a2a1072dc8ffff 85
8a2a1072dc97fff 57
8a2a1072dc9ffff 153
8a2a1072dca7fff 79
8a2a1072dcaffff 393
8a2a1072dcb7fff 9
8a2a1072dcc7fff 10
8a2a1072dcd7fff 6
8a2a1072dcdffff 2
8a2a1072dce7fff 173
8a2a1072dcf7fff 137
8a2a1072dd07fff 122
8a2a1072dd0ffff 174
8a2a1072dd17fff 17
8a2a1072dd1ffff 50
8a2a1072dd27fff 94
8a2a1072dd2ffff 89
8a2a1072dd37fff 47
8a2a1072dd47fff 695
8a2a

8a2a10746587fff 6
8a2a1074658ffff 17
8a2a10746597fff 26
8a2a107465a7fff 32
8a2a107465affff 10
8a2a107465b7fff 4
8a2a10746607fff 18
8a2a1074660ffff 13
8a2a10746617fff 20
8a2a1074661ffff 61
8a2a10746627fff 66
8a2a1074662ffff 6
8a2a10746637fff 34
8a2a10746647fff 31
8a2a1074664ffff 17
8a2a10746657fff 20
8a2a1074665ffff 8
8a2a10746667fff 79
8a2a1074666ffff 12
8a2a10746677fff 19
8a2a10746687fff 48
8a2a1074668ffff 11
8a2a10746697fff 33
8a2a1074669ffff 10
8a2a107466a7fff 27
8a2a107466affff 6
8a2a107466b7fff 69
8a2a107466c7fff 50
8a2a107466cffff 4
8a2a107466d7fff 10
8a2a107466dffff 19
8a2a107466e7fff 35
8a2a107466effff 10
8a2a107466f7fff 45
8a2a10746707fff 30
8a2a1074670ffff 88
8a2a10746717fff 4
8a2a1074671ffff 16
8a2a10746727fff 24
8a2a1074672ffff 23
8a2a10746737fff 24
8a2a10746747fff 106
8a2a1074674ffff 12
8a2a10746757fff 10
8a2a1074675ffff 171
8a2a10746767fff 39
8a2a1074676ffff 28
8a2a10746777fff 20
8a2a10746787fff 5
8a2a1074678ffff 42
8a2a10746797fff 6
8a2a1074679ffff 14
8a2a107467a7fff 5
8

8a2a107626e7fff 64
8a2a107626effff 11
8a2a107626f7fff 8
8a2a1076270ffff 99
8a2a10762717fff 47
8a2a1076271ffff 14
8a2a10762727fff 36
8a2a1076272ffff 3
8a2a10762737fff 190
8a2a10762747fff 38
8a2a1076274ffff 20
8a2a10762757fff 4
8a2a1076275ffff 2
8a2a10762767fff 531
8a2a1076276ffff 244
8a2a10762777fff 20
8a2a10762787fff 1
8a2a1076278ffff 109
8a2a10762797fff 33
8a2a1076279ffff 12
8a2a107627a7fff 185
8a2a107627affff 22
8a2a107627b7fff 30
8a2a10762807fff 38
8a2a1076280ffff 68
8a2a10762817fff 8
8a2a1076281ffff 92
8a2a1076282ffff 38
8a2a10762837fff 1304
8a2a10762847fff 24
8a2a1076284ffff 11
8a2a10762857fff 306
8a2a1076285ffff 43
8a2a10762867fff 16
8a2a1076286ffff 12
8a2a10762877fff 43
8a2a10762887fff 145
8a2a1076288ffff 151
8a2a10762897fff 22
8a2a1076289ffff 49
8a2a107628a7fff 42
8a2a107628affff 17
8a2a107628b7fff 174
8a2a107628c7fff 41
8a2a107628cffff 602
8a2a107628d7fff 286
8a2a107628dffff 61
8a2a107628e7fff 53
8a2a107628effff 154
8a2a107628f7fff 86
8a2a10762907fff 46
8a2a1076290ffff 1016
8a

8a2a10770d1ffff 23
8a2a10770d27fff 37
8a2a10770d2ffff 22
8a2a10770d37fff 17
8a2a10770d57fff 17
8a2a10770d5ffff 5
8a2a10770d77fff 6
8a2a10770d8ffff 5
8a2a10770d9ffff 1
8a2a10770daffff 33
8a2a10770db7fff 2
8a2a10771007fff 2
8a2a1077100ffff 10
8a2a10771017fff 1
8a2a10771027fff 7
8a2a1077102ffff 3
8a2a10771047fff 3
8a2a1077104ffff 7
8a2a10771057fff 1
8a2a1077105ffff 1
8a2a10771067fff 12
8a2a1077106ffff 7
8a2a10771077fff 6
8a2a10771087fff 12
8a2a1077108ffff 3
8a2a10771097fff 12
8a2a1077109ffff 3
8a2a107710a7fff 1
8a2a107710affff 1
8a2a107710b7fff 9
8a2a107710cffff 1
8a2a107710d7fff 3
8a2a107710dffff 3
8a2a107710effff 1
8a2a10771107fff 34
8a2a10771117fff 2
8a2a1077111ffff 2
8a2a10771127fff 20
8a2a1077112ffff 2
8a2a10771137fff 15
8a2a10771147fff 1
8a2a1077114ffff 2
8a2a10771167fff 17
8a2a1077116ffff 53
8a2a10771177fff 5
8a2a10771187fff 1
8a2a1077118ffff 2
8a2a10771197fff 3
8a2a1077119ffff 18
8a2a107711affff 6
8a2a107711b7fff 3
8a2a10771207fff 14
8a2a10771227fff 18
8a2a1077122ffff 4
8a2a107712

In [5]:
#automatically upload all csv to s3
import boto3, os